In [2]:
%pylab widget

from glob import glob
import os
import pandas
import json_object as jso
import pickle
import re
import scipy.signal as sig
import pandas as pd
import seaborn as sns

from pypevoc.Heterodyne import HeterodyneHarmonic
from pypevoc.PVAnalysis import PV


Populating the interactive namespace from numpy and matplotlib


In [61]:
#csvfile = '/Users/goios/Data/clarinet_simulations/20200505-highQclarino/simulation_analysis_20200505-highQclarino.csv'
csvfile = '/Users/goios/Data/clarinet_simulations/20200506-hiqhQreed_vtlosses/simulation_analysis_20200506-hiqhQreed_vtlosses.csv'
datadir = os.path.join(*os.path.split(csvfile)[:-1])
resdf = pd.read_csv(csvfile)
filelist = resdf.filename.tolist()


In [62]:
pblowu=sorted(resdf['blowing pressure'].unique())
pblowu

[1944.0, 2160.0, 2268.0, 2376.0, 2592.0]

In [63]:
tradu=sorted(resdf['tongue radius'].unique())
tradu

[0.0015, 0.003, 0.004, 0.006, 0.008, 0.01, 0.015, 0.02]

In [64]:
partdf = resdf[resdf['tongue radius']==tradu[2]]

In [86]:
figure()
sns.scatterplot(data=partdf,x='vocal tract length',y='sus_f0_avg',hue='blowing pressure')
bp = partdf['blowing pressure'].unique()[0]
zdf = partdf[partdf['blowing pressure']==bp].sort_values('vocal tract length')
plot(zdf['vocal tract length'],zdf['z_bore_pk_f_0'],label='bore peak')
plot(zdf['vocal tract length'],zdf['z_vocal_pk_f_0'], label='vt peak')
legend(loc='upper left',bbox_to_anchor=(1,1))
ylabel('Playing frequency (quiescent, Hz)')
#ylim(320,440)
tight_layout()
ylim(250,1000)

/Users/goios/Envs/unsw/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

(250, 1000)

In [82]:
figure()
sns.scatterplot(data=partdf,x='vocal tract length',y=partdf['trans_f0_avg']/partdf['sus_f0_avg'],hue='blowing pressure')
ylim(.8,1.2)

/Users/goios/Envs/unsw/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

(0.8, 1.2)

In [69]:
figure()
sns.scatterplot(data=partdf,x='vocal tract length',y='rate_1_trans',hue='blowing pressure')
#ylim(320,400)
ylabel('exp. rate (dB/s)')
legend(loc='upper left',bbox_to_anchor=(1,1))

FigureCanvasNbAgg()

In [81]:
fig,ax = subplots(2, sharex=True, sharey=True)
sns.scatterplot(data=partdf,x='vocal tract length',y='hrat_1_sus_abs_avg',hue='blowing pressure',ax=ax[0],legend=False)
ax[0].set_ylabel('quiesent pvt/pb (H1)')
sns.scatterplot(data=partdf,x='vocal tract length',y='hrat_1_trans_abs_avg',hue='blowing pressure',ax=ax[1])
ax[1].set_ylabel('tranient pvt/pb (H1)')
ax[0].set_ylim(0,2)
ax[1].legend(loc='center left',bbox_to_anchor=(1,1))
tight_layout()

FigureCanvasNbAgg()

In [71]:
fig,ax = subplots(2, sharex=True, sharey=True)
sns.scatterplot(data=partdf,x='vocal tract length',y='bore_1_sus_abs_avg',hue='blowing pressure',legend=False,ax=ax[0])
ax[0].set_ylabel('H1 bore pressure')
sns.scatterplot(data=partdf,x='vocal tract length',y='vt_1_sus_abs_avg',hue='blowing pressure',ax=ax[1])
ax[1].set_ylabel('H1 vt pressure')
ax[1].legend(loc='center left',bbox_to_anchor=(1,1))

FigureCanvasNbAgg()

In [72]:
zdict={}
nzfft=2**14
for irow, row in partdf[partdf['blowing pressure']==pblowu[0]].iterrows():
    with open(row.filename,'rb') as fo:
        pckl = pickle.load(fo)
        
    impresp_b = pckl['impresp_b']
    impresp_vt = pckl['impresp_vt']
    
    f = np.arange(nzfft)/nzfft*sr

    rf=np.fft.fft(impresp_vt,n=nzfft)
    z_vt=(1+rf)/(1-rf)*pckl['zch_vt']


    rf=np.fft.fft(impresp_b,n=nzfft)
    z_b=(1+rf)/(1-rf)*pckl['zch_b']
    
    zdict[row['vocal tract length']]={'z_vt': z_vt, 'z_b':z_b}
    
#     z_b_pks,_ = sig.find_peaks(np.abs(z_b))
#     for ii,zz in enumerate(z_b_pks[:nzpks]):
#         results['z_bore_pk_f_{}'.format(ii)] = f[zz]
#         results['z_bore_pk_z_{}'.format(ii)] = z_b[zz]
#     z_v_pks,_ = sig.find_peaks(np.abs(z_vt))
#     for ii,zz in enumerate(z_v_pks[:nzpks]):
#         results['z_vocal_pk_f_{}'.format(ii)] = f[zz]
#         results['z_vocal_pk_z_{}'.format(ii)] = z_vt[zz]
        


In [77]:
fig,ax=subplots(2,sharex=True)
every = 5
for ii,(k,v) in enumerate(zdict.items()):
    if ii%every == 0:
        ax[0].semilogy(f,np.abs(v['z_vt']),lw=1)
        ax[1].plot(f,np.angle(v['z_vt']),lw=1)
    
ax[0].semilogy(f,np.abs(z_b),color='k')
ax[1].plot(f,np.angle(z_b),color='k')

xlim(0,2000)

FigureCanvasNbAgg()

(0, 2000)

In [74]:
fig,ax=subplots(2,sharex=True)
for k,v in zdict.items():
    ax[0].semilogy(f,np.abs(v['z_vt'] +z_b),lw=1)
    ax[1].plot(f,np.angle(v['z_vt']+z_b),lw=1)
    
xlim(500,1400)

FigureCanvasNbAgg()

(500, 1400)

In [80]:


tsigdf = partdf[partdf['blowing pressure']==pblowu[4]].sort_values('vocal tract length').iloc[::5]
nplots = len(tsigdf)
nrows=3
ncols = int(np.ceil(nplots/nrows))

fig,ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize=(8,6))
axf = ax.flatten()


for ii,(_,row) in enumerate(tsigdf.iterrows()):
    thisfile = row['filename']
    with open(thisfile,'rb') as fo:
        pckl = pickle.load(fo)
    axf[ii].plot(pckl['p_b'])
    axf[ii].plot(pckl['p_vt'])#+row['p_blow'])
    axf[ii].set_title('{:.3f}'.format(row['vocal tract length']))
        
        


FigureCanvasNbAgg()

In [57]:
js['environment/reed'].to_python()

{'_comment': "reed moves instantaneously if 'dynamic' is false, that means res. freq and q are not taken into account",
 'stiffness': 500000000.0,
 'resonance frequency': 1500,
 'quality factor': 2.0,
 'rest opening': 1e-05,
 'dynamic': True,
 'non-linear force': {'_comment': 'non-linear force kicks in when x/x0 > onset_opening fraction and is exponential like exp((x-xth)/(xc-xth))',
  '_choices_model': ['disabled', 'inverse_p', 'log'],
  'model': 'inverse_p',
  'onset pressure fraction': 0.9}}

In [35]:
m=js['environment/reed/stiffness']/(js['environment/reed/resonance frequency']*2*pi)**2
r = js['environment/reed/quality factor']*np.sqrt(m/js['environment/reed/stiffness'])

print('m = {:.3e} kg/m2'.format(m))
print('r = {:.3e} kg/s/m2'.format(r))
print('r = {:.3e} kg/s2/m2 (Pa/m2)'.format(js['environment/reed/stiffness']))

m = 5.629e+00 kg/m2
r = 2.122e-04 kg/s/m2
r = 5.000e+08 kg/s2/m2 (Pa/m2)


In [57]:
tsust = .1
nsust_smpl = int(tsust*sr)

p_b = pckl['p_b']
p_vt = pckl['p_vt']
pvoc = PV(p_b,sr=sr,progress=False)
pvoc.run_pv()
fig,ax = subplots(2,sharex=True)
idx = (pvoc.fundamental_frequency>50) & (pvoc.fundamental_magnitude>np.max(pvoc.fundamental_magnitude)/1000)
ax[0].plot(pvoc.t[idx],pvoc.fundamental_frequency[idx])
ax[1].semilogy(pvoc.t,pvoc.fundamental_magnitude)
ax[0].axhline(f0,color='blue')

/Users/goios/Devel/pypevoc/pypevoc/PVAnalysis.py:171: RuntimeWarning: divide by zero encountered in true_divide
  frat = fx / self.oldfft
/Users/goios/Devel/pypevoc/pypevoc/PVAnalysis.py:171: RuntimeWarning: invalid value encountered in true_divide
  frat = fx / self.oldfft


FigureCanvasNbAgg()

NameError: name 'f0' is not defined

In [58]:
isus = pvoc.t>(pvoc.t[-1]-tsust)
fsus = pvoc.fundamental_frequency[isus]
print(np.mean(fsus),np.std(fsus))
asus = pvoc.fundamental_magnitude[isus]
print(np.mean(asus),np.std(asus),np.polyfit(pvoc.t[isus],asus,1))

283.59936028124963 1.3709176654205104
2779.8760170330343 13.014954922808394 [  90.7877076  2748.52399534]


In [59]:
results={}

amax = max(pvoc.fundamental_magnitude)

a_trans_end = amax*.9
a_trans_start = amax*.01
i_trans_end = np.flatnonzero(pvoc.fundamental_magnitude>a_trans_end)[0]-1
i_trans_start = np.flatnonzero(pvoc.fundamental_magnitude[:i_trans_end]<a_trans_start)[-1]
i_trans = slice(i_trans_start,i_trans_end)
atrans = 20*np.log10(pvoc.fundamental_magnitude[i_trans])
ttrans = pvoc.t[i_trans]
polytrans = np.polyfit(ttrans,atrans,1) 
print(polytrans)
fig,ax = subplots(1)
ax.plot(pvoc.t,20*np.log10(pvoc.fundamental_magnitude))
ax.plot(pvoc.t[i_trans],np.polyval(polytrans,pvoc.t[i_trans]))
ax.axvspan(pvoc.t[i_trans_start],pvoc.t[i_trans_end],color='red',alpha=.2)

fsusav = np.mean(fsus)
print('sust','f0',fsusav,np.std(fsus))
print('trns','f0',np.mean(pvoc.fundamental_frequency[i_trans]),np.std(pvoc.fundamental_frequency[i_trans]))
results['sus_f0_avg']=fsusav
results['sus_f0_std']=np.std(fsus)
results['trans_f0_avg']=np.mean(pvoc.fundamental_frequency[i_trans])
results['trans_f0_std']=np.std(pvoc.fundamental_frequency[i_trans])


hhb = HeterodyneHarmonic(p_b,sr=sr,nwind=1024*2,nhop=256*2,f=fsusav)
hhv = HeterodyneHarmonic(p_vt,sr=sr,nwind=1024*2,nhop=256*2,f=fsusav)

for ii in range(3):
    ax.plot(hhb.t,20*np.log10(np.abs(hhb.camp[:,ii])))
for ii in range(3):
    ax.plot(hhv.t,20*np.log10(np.abs(hhv.camp[:,ii])),ls='--')

t_trans_start = pvoc.t[i_trans_start]
t_trans_end = pvoc.t[i_trans_end]


hidxsus = hhb.t>(hhb.t[-1]-tsust)
hidxtrans = (hhb.t>=t_trans_start) & (hhb.t<=t_trans_end)

for ii in range(3):
    print('harm ',ii+1)
    for hlab,hh in [('bore',hhb),('vt',hhv)]:
        print(' '*2,hlab)
        v = np.abs(hh.camp[:,ii])
        amax = max(v)
        a_trans_end = amax*.9
        a_trans_start = amax*.01
        i_trans_end = np.flatnonzero(v>a_trans_end)[0]-1
        i_trans_start = np.flatnonzero(v[:i_trans_end]<a_trans_start)[-1]
        print(' '*4,'trans times',hh.t[i_trans_start],hh.t[i_trans_end])
        results['{}_{}_trans_start'.format(hlab,ii)]=hh.t[i_trans_start]
        results['{}_{}_trans_end'.format(hlab,ii)]=hh.t[i_trans_end]

        hidxtrans = slice(i_trans_start,i_trans_end)

        for rlab, reg in [('sus',hidxsus),('trans',hidxtrans)]:
            
            v = hh.camp[reg,ii]
            print(' '*4,rlab,'abs',np.mean(np.abs(v)),np.std(np.abs(v)))
            print(' '*4,rlab,'arg',np.mean(np.unwrap(np.angle(v))),np.std(np.unwrap(np.angle(v))))
            results['{}_{}_{}_abs_avg'.format(hlab,ii+1,rlab)] = np.mean(np.abs(v))
            results['{}_{}_{}_abs_std'.format(hlab,ii+1,rlab)] = np.std(np.abs(v))
            results['{}_{}_{}_arg_avg'.format(hlab,ii+1,rlab)] = np.mean(np.unwrap(np.angle(v)))
            results['{}_{}_{}_arg_std'.format(hlab,ii+1,rlab)] = np.std(np.unwrap(np.angle(v)))

            v = hhv.camp[reg,ii]/hhb.camp[reg,ii]
            print(' '*4,rlab,'rat','abs',np.mean(np.abs(v)),np.std(np.abs(v)))
            print(' '*4,rlab,'rat','arg',np.mean(np.unwrap(np.angle(v))),np.std(np.unwrap(np.angle(v))))
            results['{}_{}_{}_arg_avg'.format('hrat',ii+1,rlab)] = np.mean(np.unwrap(np.angle(v)))
            results['{}_{}_{}_arg_std'.format('hrat',ii+1,rlab)] = np.std(np.unwrap(np.angle(v)))
            results['{}_{}_{}_abs_avg'.format('hrat',ii+1,rlab)] = np.mean(np.abs(v))
            results['{}_{}_{}_abs_std'.format('hrat',ii+1,rlab)] = np.std(np.abs(v))

        v = hh.camp[reg,ii]
        trpol = np.polyfit(hh.t[reg],20*np.log10(np.abs(v)),1)
        print(' '*4,rlab,'exr',trpol[0])


[900.02001904  -5.71868608]


FigureCanvasNbAgg()

sust f0 283.59936028124963 1.3709176654205104
trns f0 287.1470921584932 3.897673964146086
harm  1
   bore
     trans times 0.037333333333333336 0.08266666666666667
     sus abs 2816.8611353407787 0.43065570347012655
     sus arg 1.407651082580158 0.004424375805931017
     sus rat abs 0.017759088125692827 0.0001266599070545336
     sus rat arg 2.8063067492185256 0.007338305248958317
     trans abs 505.9838335855123 585.1000191045871
     trans arg 1.4666387074135556 0.12131611660008845
     trans rat abs 0.04749181935665465 0.002128258814139788
     trans rat arg 2.2809463402327688 0.029964904575219328
     trans exr 900.0997849913524
   vt
     trans times 0.029333333333333333 0.07466666666666667
     sus abs 50.024887939780854 0.3572262513299403
     sus arg -2.069227475380903 0.008766596221130104
     sus rat abs 0.017759088125692827 0.0001266599070545336
     sus rat arg 2.8063067492185256 0.007338305248958317
     trans abs 10.800522809202166 12.491827950017509
     trans arg -2.48

/Users/goios/Envs/unsw/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log10
  from ipykernel import kernelapp as app


In [60]:
fig,ax = subplots(2,sharex=True)

hrat = hhv.camp/hhb.camp
for ii in range(4):
    ax[0].plot(hhb.t,20*np.log10(np.abs(hrat[:,ii])))
    ax[1].plot(hhb.t,np.angle(hrat[:,ii]))

FigureCanvasNbAgg()

In [6]:
df.columns

Index(['tongue_rad', 'vt_len', 'p_blow', 'filename'], dtype='object')

In [193]:
tr=sorted(df['tongue_rad'].unique())
tr

[0.0015, 0.003, 0.004, 0.006, 0.008, 0.01, 0.015, 0.02]

In [194]:
pba=sorted(df.p_blow.unique())
pba

[1944.0, 2160.0, 2268.0, 2376.0, 2592.0]

In [199]:
fl=df[(df['p_blow']==pba[2])&(df.tongue_rad==tr[2])].sort_values('vt_len').iloc[::5]

In [200]:
tr[2]

0.004

In [201]:
%matplotlib widget

nplots = len(fl)
nrows=3
ncols = int(np.ceil(nplots/nrows))

fig,ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize=(8,6))
axf = ax.flatten()


for ii,(_,row) in enumerate(fl.iterrows()):
    thisfile = row['filename']
    with open(thisfile,'rb') as fo:
        pckl = pickle.load(fo)
    axf[ii].plot(pckl['p_b'])
    axf[ii].plot(pckl['p_vt'])#+row['p_blow'])
    axf[ii].set_title('{:.3f}'.format(row['vt_len']))

FigureCanvasNbAgg()

In [198]:
fl

,tongue_rad,vt_len,p_blow,filename
677,0.004,0.159360,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
257,0.004,0.161166,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
32,0.004,0.162972,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
941,0.004,0.164778,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
1029,0.004,0.166584,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
1056,0.004,0.168390,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
544,0.004,0.170196,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
1066,0.004,0.172002,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
1256,0.004,0.173808,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
40,0.004,0.175614,2268.0,/Users/goios/Data/clarinet_simulations/2020050...
